# Eine Startseite für das Dashboard gestalten

## Story
Amir möchte sich zunächst einen schnellen Überblick verschaffen: Wie werden Bäume in Berlin gegossen, und wie engagieren sich die Bürger:innen dabei? Bei seiner Recherche stößt er auf die Plattform *Gieß den Kiez*. Besonders beeindruckt ihn, wie anschaulich die Daten dort visualisiert sind – das möchte er für seine eigene R-Shiny-Anwendung übernehmen.

## Lernziele
- Der Aufbau eines Dashboards als Form der Visualisierung in der Verwaltungswissenschaft
- Gestaltung einer übersichtlichen Startseite für ein R-Shiny-Dashboard
- Zentrale Informationen klar strukturiert darstellen

## Bibliotheken laden

**Hinweis**: Um diesen Code auszuführen, müssen Sie R und die erforderlichen Pakete installiert haben.

In [1]:
# WICHTIG: Dieser Code-Block dient nur zur Demonstration der benötigten Bibliotheken
install.packages(c("shiny", "shinydashboard", 
                   "leaflet", "ggplot2", "dplyr", 
                   "lubridate", "plotly", "tidyr", 
                   "stringr", "leaflet.extras", "shinyBS"))

library(shiny)
library(leaflet)
library(ggplot2)
library(dplyr)
library(lubridate)
library(shinydashboard)
library(plotly)
library(leaflet.extras)
library(tidyr)
library(stringr)
library(shinyBS)

# Daten laden (passen Sie den Pfad an)
# df_merged <- read.csv("pfad/zu/ihren/daten.csv")

Warning message in install.packages(c("shiny", "shinydashboard", "leaflet", "ggplot2", :
"'lib = "C:/Program Files/R/R-4.4.2/library"' is not writable"


ERROR: Error in install.packages(c("shiny", "shinydashboard", "leaflet", "ggplot2", : unable to install packages


## Benutzeroberfläche (UI)

Die Benutzeroberfläche besteht aus zwei Teilen:
- einer Seitenleiste (`sidebarMenu`) mit der Navigation
- einem Inhaltsbereich (`tabItem`) mit ValueBoxen und Dropdowns

### Seitenleiste mit Navigation

```r
# DEMO: Seitenleiste (läuft nicht eigenständig)
# Dies ist ein Code-Beispiel zur Illustration

dashboardSidebar(
  sidebarMenu(
    menuItem("Startseite", tabName = "start", icon = icon("home"))
  )
)
```

**Erklärung**:
- `sidebarMenu()` erzeugt die Navigationsleiste
- `menuItem()` erstellt einen Menüpunkt mit:
  - `"Startseite"` als sichtbarer Text
  - `tabName = "start"` zur Verknüpfung mit dem Inhaltsbereich
  - `icon("home")` für ein Symbol

### Inhaltsbereich mit ValueBoxen

```r
# DEMO: Inhaltsbereich (läuft nicht eigenständig)

tabItems(
  tabItem(
    tabName = "start",
    box(
      title = "Overview",
      status = "primary",
      solidHeader = TRUE,
      width = 12,
      
      fluidRow(
        valueBoxOutput("total_trees", width = 6),
        valueBoxOutput("total_tree_watered", width = 6)
      ),
      
      fluidRow(
        column(
          width = 6,
          selectInput(
            "bezirk",
            "Bezirk auswählen:",
            choices = c("Alle", unique(df_merged$bezirk)),
            selected = "Alle",
            multiple = TRUE
          )
        )
      )
    )
  )
)
```

## Server-Logik

Der Server beobachtet Eingabefelder (`input$...`) und aktualisiert automatisch die Ausgaben (`output$...`).

### Hilfsfunktionen für Einheiten-Umrechnung

In [ ]:
# Diese Funktionen können eigenständig getestet werden

convert_units <- function(liters) {
  if (liters >= 1e6) {
    return(list(value = round(liters / 1e6, 2), unit = "ML"))
  } else if (liters >= 1e3) {
    return(list(value = round(liters / 1e3, 2), unit = "m³"))
  } else {
    return(list(value = round(liters, 2), unit = "L"))
  }
}

full_unit <- function(unit) {
  switch(unit,
    "ML" = "Mega Liter", 
    "m³" = "Kubikmeter", 
    "L" = "Liter",
    unit
  )
}

# Test der Funktionen
convert_units(1500)
full_unit("ML")

## Reaktive Datenfilterung

```r
# DEMO: Reaktive Funktion (läuft nur innerhalb einer Shiny App)

filteredData <- reactive({
  req(input$bezirk)
  
  df <- df_merged
  df_filtered <- df
  
  # Filter nach Bezirk
  if (!("Alle" %in% input$bezirk)) {
    df_filtered <- df_filtered %>% filter(bezirk %in% input$bezirk)
  }
  
  df_filtered
})
```

**Erklärung**:
- `reactive({...})` erzeugt eine reaktive Funktion, die automatisch neu berechnet wird
- `req(input$bezirk)` sorgt dafür, dass die Funktion nur ausgeführt wird, wenn Eingaben vorhanden sind
- Die Filterlogik passt die Daten basierend auf der Bezirksauswahl an

## ValueBoxes erstellen

```r
# DEMO: ValueBox für Gesamtzahl der Bäume

output$total_trees <- renderValueBox({
  valueBox(
    formatC(n_distinct(df_merged$gisid), format = "d", big.mark = "."),
    "Gesamtzahl der Bäume",
    icon = icon("tree"),
    color = "green"
  )
})

# DEMO: ValueBox für gegossene Bäume

output$total_tree_watered <- renderValueBox({
  valueBox(
    formatC(
      n_distinct(filteredData()$gisid[!is.na(filteredData()$timestamp)]), 
      format = "d", big.mark = "."
    ),
    "Gesamtzahl der gegossenen Bäume",
    icon = icon("tint"),
    color = "blue"
  )
})
```

## Komplette Shiny App

**Um die vollständige App auszuführen, müssen Sie:**
1. Ihre Daten laden (df_merged)
2. Den gesamten Code in einem R-Skript oder einer R-Umgebung ausführen
3. Die App wird in einem separaten Fenster/Browser geöffnet

In [ ]:
# VOLLSTÄNDIGE APP - Ausführen in R/RStudio

library(shiny)
library(shinydashboard)
library(dplyr)

# Daten laden (Beispiel - anpassen!)
# df_merged <- read.csv("ihre_daten.csv")

# UI-Definition
ui <- dashboardPage(
  dashboardHeader(title = "Gieß den Kiez Dashboard"),
  dashboardSidebar(
    sidebarMenu(
      menuItem("Startseite", tabName = "start", icon = icon("home"))
    )
  ),
  dashboardBody(
    tabItems(
      tabItem(tabName = "start",
              box(title = "Overview", status = "primary", solidHeader = TRUE, width = 12,
                  fluidRow(
                    valueBoxOutput("total_trees", width = 6),
                    valueBoxOutput("total_tree_watered", width = 6)
                  ),
                  fluidRow(
                    column(width = 6,
                           selectInput("bezirk", "Bezirk auswählen:", 
                                       choices = c("Alle"), 
                                       selected = "Alle", multiple = TRUE)
                    )
                  )
              )
      )
    )
  )
)

# Server-Logik
server <- function(input, output, session) {
  
  convert_units <- function(liters) {
    if (liters >= 1e6) {
      return(list(value = round(liters / 1e6, 2), unit = "ML"))
    } else if (liters >= 1e3) {
      return(list(value = round(liters / 1e3, 2), unit = "m³"))
    } else {
      return(list(value = round(liters, 2), unit = "L"))
    }
  }
  
  full_unit <- function(unit) {
    switch(unit,
           "ML" = "Mega Liter", 
           "L" = "Liter", 
           "m³" = "Kubikmeter",
           unit)
  }
  
  filteredData <- reactive({
    req(input$bezirk)
    
    # df <- df_merged
    # df_filtered <- df
    # 
    # if (!("Alle" %in% input$bezirk)) {
    #   df_filtered <- df_filtered %>% filter(bezirk %in% input$bezirk)
    # }
    # 
    # df_filtered
    
    # Dummy-Daten für Demo
    data.frame(gisid = 1:100, timestamp = Sys.time())
  })
  
  output$total_trees <- renderValueBox({
    valueBox(
      # formatC(n_distinct(df_merged$gisid), format = "d", big.mark = "."),
      "50.000",  # Dummy-Wert
      "Gesamtzahl der Bäume",
      icon = icon("tree"),
      color = "green"
    )
  })
  
  output$total_tree_watered <- renderValueBox({
    valueBox(
      # formatC(n_distinct(filteredData()$gisid[!is.na(filteredData()$timestamp)]), 
      #         format = "d", big.mark = "."),
      "12.345",  # Dummy-Wert
      "Gesamtzahl der gegossenen Bäume",
      icon = icon("tint"),
      color = "blue"
    )
  })
}

# App starten - Entfernen Sie das # um die App zu starten
# shinyApp(ui, server)

## Übersicht der Funktionen

| Funktion/Operator | Bedeutung |
|-------------------|----------|
| `<-` | weist einer Variable einen Wert zu |
| `if (...)` / `else` | Bedingte Ausführung |
| `%in%` | prüft, ob ein Wert in einer Liste ist |
| `mean()` | Durchschnitt berechnen |
| `sum()` | Summe berechnen |
| `switch()` | wählt abhängig vom Wert einen Fall |
| `mutate()` | erzeugt oder verändert Spalten |
| `filter()` | filtert Zeilen in einem Datensatz |
| `is.na()` | prüft auf fehlende Werte |
| `n_distinct()` | zählt eindeutige Werte |
| `reactive()` | erstellt reaktive Ausdrücke |
| `renderValueBox()` | rendert ValueBoxen für Shiny |

## Wichtige Hinweise zur Ausführung

⚠️ **Shiny Apps laufen nicht direkt in Jupyter Notebooks**

Um diese App auszuführen:
1. Kopieren Sie den kompletten App-Code in ein R-Skript (`.R` Datei)
2. Laden Sie Ihre Daten (df_merged)
3. Führen Sie es in RStudio oder R aus: `source("app.R")` oder klicken Sie auf "Run App"
4. Die App öffnet sich in einem Browser-Fenster

Dieses Notebook dient als **Tutorial und Dokumentation** der einzelnen Komponenten.

### Alternative: R Markdown

Für interaktive R-Dokumente mit ausführbarem Code empfiehlt sich R Markdown (`.Rmd`) statt Jupyter Notebooks.

## Leitfrage und Ausblick

Die zentrale Leitfrage von Amirs Fallstudie lautet: **Wo ist das höchste Bürgerengagement?**

Mit den Daten aus Gieß den Kiez kann er diese Frage bereits auf der Startseite beantworten: Pro Bezirk lässt sich das Engagement direkt darstellen und vergleichen. Am meisten engagierten sich die Bürger:innen in **Mitte**, danach folgen **Tempelhof-Schöneberg** und **Charlottenburg-Wilmersdorf**.

### Weiterführende Fragen:

- Wo treten die höchsten Ausprägungen des Bürgerengagements auf?
- Welche zusätzlichen Datensätze lassen sich einbeziehen, um die Analyse zu vertiefen?
- Wie können interaktive Dashboards diese Faktoren verständlich und vergleichbar darstellen?